In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 1. Generate Random Rectangles
def generate_rectangles(n=30, min_s=5, max_s=25):
    rects = []
    for i in range(n):
        w = random.randint(min_s, max_s)
        h = random.randint(min_s, max_s)
        rects.append({'id': i, 'w': w, 'h': h})
    return rects

rectangles = generate_rectangles(40)

## Implementation: Guillotine Packer

We maintain a list of `free_rects`. When we place an item, we remove the used free rect and add two new ones created by the split.

In [ ]:
class GuillotinePacker:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        # Free rects stored as (x, y, w, h)
        self.free_rects = [(0, 0, width, height)]
        self.packed_items = []

    def pack(self, rects):
        # Sort by area (descending) often gives better results than height
        sorted_rects = sorted(rects, key=lambda r: r['w'] * r['h'], reverse=True)
        
        for rect in sorted_rects:
            self.place_rect(rect)
            
        return self.packed_items

    def place_rect(self, rect):
        # Find best free rect (Best Area Fit)
        best_idx = -1
        min_leftover_area = float('inf')
        best_split_horizontal = True # Track best split direction
        
        for i, free in enumerate(self.free_rects):
            fx, fy, fw, fh = free
            
            # Check if fits
            if rect['w'] <= fw and rect['h'] <= fh:
                # Calculate leftover area if we place it here
                leftover = (fw * fh) - (rect['w'] * rect['h'])
                
                if leftover < min_leftover_area:
                    min_leftover_area = leftover
                    best_idx = i
                    # Heuristic: Split along the shorter axis to minimize narrow strips
                    # If remaining width < remaining height, split horizontally (cut vertical line)
                    rem_w = fw - rect['w']
                    rem_h = fh - rect['h']
                    best_split_horizontal = (rem_w < rem_h)
                    
        # If we found a spot
        if best_idx != -1:
            # Get the free rect
            fx, fy, fw, fh = self.free_rects.pop(best_idx)
            
            # Place item
            rect['x'] = fx
            rect['y'] = fy
            self.packed_items.append(rect)
            
            rw, rh = rect['w'], rect['h']
            
            # Create two new free rects based on split direction
            if best_split_horizontal:
                # Split Horizontally (Cut Vertical Line)
                # Bottom-Right: x=fx+rw, y=fy, w=fw-rw, h=rh
                # Top:          x=fx, y=fy+rh, w=fw, h=fh-rh
                new_right = (fx + rw, fy, fw - rw, rh)
                new_top   = (fx, fy + rh, fw, fh - rh)
            else:
                # Split Vertically (Cut Horizontal Line)
                # Right:        x=fx+rw, y=fy, w=fw-rw, h=fh
                # Top-Left:     x=fx, y=fy+rh, w=rw, h=fh-rh
                new_right = (fx + rw, fy, fw - rw, fh)
                new_top   = (fx, fy + rh, rw, fh - rh)
            
            # Add valid new free rects
            if new_right[2] > 0 and new_right[3] > 0:
                self.free_rects.append(new_right)
            if new_top[2] > 0 and new_top[3] > 0:
                self.free_rects.append(new_top)
                
            # Optimization: Merge free rects? (Skipped for simplicity, but important for advanced versions)

# Run
bin_size = 100
packer = GuillotinePacker(bin_size, bin_size)
packed = packer.pack(rectangles)

print(f"Packed {len(packed)} / {len(rectangles)} rectangles.")

## Visualization
Notice how the layout is not defined by strict rows. Items are tucked into available corners.

In [ ]:
def plot_guillotine(packed_items, free_rects, bin_size):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Draw Bin
    ax.add_patch(patches.Rectangle((0, 0), bin_size, bin_size, linewidth=2, edgecolor='black', facecolor='none'))
    
    # Draw Packed Items
    colors = plt.cm.get_cmap('viridis', len(packed_items))
    for i, rect in enumerate(packed_items):
        r = patches.Rectangle((rect['x'], rect['y']), rect['w'], rect['h'], 
                            linewidth=1, edgecolor='black', facecolor=colors(i), alpha=0.8)
        ax.add_patch(r)
        ax.text(rect['x'] + rect['w']/2, rect['y'] + rect['h']/2, str(rect['id']), 
                ha='center', va='center', fontsize=8, color='white')

    # Draw Remaining Free Rects (Ghosted)
    for fr in free_rects:
        fx, fy, fw, fh = fr
        r = patches.Rectangle((fx, fy), fw, fh, 
                            linewidth=1, edgecolor='red', facecolor='none', linestyle='--', alpha=0.3)
        ax.add_patch(r)

    ax.set_xlim(-5, bin_size + 5)
    ax.set_ylim(-5, bin_size + 5)
    ax.set_aspect('equal')
    plt.title("Guillotine Packing (Best Area Fit)")
    plt.show()

plot_guillotine(packer.packed_items, packer.free_rects, bin_size)